In [71]:
import cvxpy as cp
import numpy as np
import pandas as pd 

In [19]:
df = pd.read_csv('/home/fergus/data/mldatasets/energy_data/energydata_complete.csv')
y = df.Appliances.to_numpy()
df = df.drop(columns=['date', 'lights', 'Appliances', 'rv1', 'rv2'])
df

,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,...,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint
0,19.890000,47.596667,19.200000,44.790000,19.790000,44.730000,19.000000,45.566667,17.166667,55.200000,...,18.2000,48.900000,17.033333,45.5300,6.600000,733.5,92.000000,7.000000,63.000000,5.300000
1,19.890000,46.693333,19.200000,44.722500,19.790000,44.790000,19.000000,45.992500,17.166667,55.200000,...,18.2000,48.863333,17.066667,45.5600,6.483333,733.6,92.000000,6.666667,59.166667,5.200000
2,19.890000,46.300000,19.200000,44.626667,19.790000,44.933333,18.926667,45.890000,17.166667,55.090000,...,18.2000,48.730000,17.000000,45.5000,6.366667,733.7,92.000000,6.333333,55.333333,5.100000
3,19.890000,46.066667,19.200000,44.590000,19.790000,45.000000,18.890000,45.723333,17.166667,55.090000,...,18.1000,48.590000,17.000000,45.4000,6.250000,733.8,92.000000,6.000000,51.500000,5.000000
4,19.890000,46.333333,19.200000,44.530000,19.790000,45.000000,18.890000,45.530000,17.200000,55.090000,...,18.1000,48.590000,17.000000,45.4000,6.133333,733.9,92.000000,5.666667,47.666667,4.900000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19730,25.566667,46.560000,25.890000,42.025714,27.200000,41.163333,24.700000,45.590000,23.200000,52.400000,...,24.7000,50.074000,23.200000,46.7900,22.733333,755.2,55.666667,3.333333,23.666667,13.333333
19731,25.500000,46.500000,25.754000,42.080000,27.133333,41.223333,24.700000,45.590000,23.230000,52.326667,...,24.7000,49.790000,23.200000,46.7900,22.600000,755.2,56.000000,3.500000,24.500000,13.300000
19732,25.500000,46.596667,25.628571,42.768571,27.050000,41.690000,24.700000,45.730000,23.230000,52.266667,...,24.7000,49.660000,23.200000,46.7900,22.466667,755.2,56.333333,3.666667,25.333333,13.266667
19733,25.500000,46.990000,25.414000,43.036000,26.890000,41.290000,24.700000,45.790000,23.200000,52.200000,...,24.6625,49.518750,23.200000,46.8175,22.333333,755.2,56.666667,3.833333,26.166667,13.233333


In [21]:
A = df.to_numpy()
A.shape

(19735, 24)

First take the dataset and stack the isntances along the rows of a matrix. In the simple linear regression problem the output y is predicted by weighting each feature. Therefore the goal is to learn $ x $ so that $ y = Ax $. 

The easiest solution is $ y= A^{-1}y = A^{-1}Ax$. However this requires $A$ to be invertible - this is clearly not possible with a rectangular matrix

In [23]:
A.shape, y.shape

((19735, 24), (19735,))

The next solution is using ordinary least squares : 

$$ x= (A^T A)^{-1} A^T b$$

$A^TA$ is psd so invertible. 

To measure the fit can do use the l2 norm on the residual: $||Ax - y||_2^2$

In [84]:
x = np.linalg.inv(A.T @ A) @ A.T @ y
value = np.linalg.norm(A @ x - y, ord=2) ** 2
value, x

(177062001.4005222,
 array([  0.3823243 ,  16.07934864, -20.28270385, -14.39113808,
         25.3971245 ,   5.0731454 ,   2.94270269,   1.92969736,
         -0.048628  ,   0.24076017,   7.68446588,   0.34940726,
          1.0795233 ,  -1.82174965,  10.22720751,  -5.90933933,
        -21.99612932,  -1.5234603 , -10.49278327,   0.15934105,
         -0.87379371,   2.17287998,   0.17570575,   4.87375311]))

<br>
This ordinary least squares solution is the analytical solution of the optimisation problem: 

\begin{align*}
\text{minimize } & ||Ax-y||_2^2 
\end{align*}

To veryify this;



In [79]:
# ValueError: Incompatible dimensions (19735, 24) (19735, 1)
x = cp.Variable(24)
obj = cp.Minimize(cp.norm(A @ x - y, 2) ** 2)
problem = cp.Problem(obj)
problem.solve(solver=cp.SCS,verbose=True)

                                     CVXPY                                     
                                     v1.3.0                                    
(CVXPY) Apr 27 09:30:05 PM: Your problem has 24 variables, 0 constraints, and 0 parameters.
(CVXPY) Apr 27 09:30:05 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Apr 27 09:30:05 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Apr 27 09:30:05 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Apr 27 09:30:05 PM: Compiling problem (target solver=SCS).
(CVXPY) Apr 27 09:30:05 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> 

177062001.40052217

In [80]:
problem.value, x.value

(177062001.40052217,
 array([  0.38232408,  16.07934861, -20.28270368, -14.39113802,
         25.39712449,   5.07314541,   2.9427027 ,   1.92969732,
         -0.0486282 ,   0.24076017,   7.68446585,   0.34940726,
          1.07952304,  -1.82174961,  10.22720758,  -5.90933937,
        -21.99612883,  -1.52346029, -10.49278323,   0.15934105,
         -0.87379372,   2.17287998,   0.17570575,   4.87375307]))

same result as before :). interestingly the default solver failed which is why i'm using SCS

<br>

# Ridge Regression 

This extension of the least squares problem adds a penalty term to regularise the weight vector $x$.

\begin{align*}
\text{minimize } & ||Ax-y||_2^2 + \beta ||x||_2^2
\end{align*}


In [102]:
# ValueError: Incompatible dimensions (19735, 24) (19735, 1)
x = cp.Variable(24)
obj = cp.Minimize(cp.norm(A @ x - y, 2)**2 + 10*(cp.norm(x, 2)**2))
#constraint = [beta >= 0.00001]
problem = cp.Problem(obj, constraint)
problem.solve(solver=cp.SCS,verbose=True)
problem.value, x.value

                                     CVXPY                                     
                                     v1.3.0                                    
(CVXPY) Apr 27 09:39:15 PM: Your problem has 25 variables, 1 constraints, and 0 parameters.
(CVXPY) Apr 27 09:39:15 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Apr 27 09:39:15 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Apr 27 09:39:15 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Apr 27 09:39:15 PM: Compiling problem (target solver=SCS).
(CVXPY) Apr 27 09:39:15 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> 

(177085829.92423603,
 array([  0.3604498 ,  16.05707142, -20.20156272, -14.35509732,
         25.3572786 ,   5.07084874,   2.92447014,   1.92301673,
         -0.07037059,   0.24103363,   7.66385249,   0.34875621,
          1.05639943,  -1.82165379,  10.21043017,  -5.90746953,
        -21.92411421,  -1.52556789, -10.44524224,   0.15834116,
         -0.86789639,   2.1734008 ,   0.17560868,   4.83820786]))

Very little difference in the optimal value and solution. I think i'll redo this at some point in the future with a different dataset. The magnitude of the loss is so big that regularisation isn't impactful. 

<br>

# Lasso Regression

This extension of the least squares problem uses the l1 norm to regularise $x$.

\begin{align*}
\text{minimize } & ||Ax-y||_2^2 + \beta ||x||_1
\end{align*}


In [104]:
# ValueError: Incompatible dimensions (19735, 24) (19735, 1)
x = cp.Variable(24)
obj = cp.Minimize(cp.norm(A @ x - y, 2)**2 + 10*(cp.norm(x, 1)))
#constraint = [beta >= 0.00001]
problem = cp.Problem(obj, constraint)
problem.solve(solver=cp.SCS,verbose=True, max_iters=1000)
problem.value, x.value

                                     CVXPY                                     
                                     v1.3.0                                    
(CVXPY) Apr 27 09:40:17 PM: Your problem has 25 variables, 1 constraints, and 0 parameters.
(CVXPY) Apr 27 09:40:17 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Apr 27 09:40:17 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Apr 27 09:40:17 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Apr 27 09:40:17 PM: Compiling problem (target solver=SCS).
(CVXPY) Apr 27 09:40:17 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> 

/home/fergus/.local/lib/python3.10/site-packages/cvxpy/problems/problem.py:1385: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


(inf, None)

Solver had trouble converging here. 